In [1]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import when

spark = SparkSession.builder.appName("CustomerOrdersJobSQL").getOrCreate()

In [2]:
customers_path = "/opt/spark-apps/input/customers.csv"
orders_path = "/opt/spark-apps/input/orders.json"
output_path_csv = "/tmp/orders_enriched_csv"
output_path_parquet = "/tmp/orders_enriched_parquet"

In [3]:
df_customers = spark.read.option("header", True).csv(customers_path)
df_orders = spark.read.json(orders_path)


In [4]:
df_customers.show()

+-----------+----------+----------------+---+-------+
|customer_id|      name|           email|age|country|
+-----------+----------+----------------+---+-------+
|        101|  John Doe|john@example.com| 28|     US|
|        102|Jane Smith|jane@example.com| 34|     UK|
|        103|  Ali Khan| ali@example.com| 40|    UAE|
|        104|Rita Mehra|rita@example.com| 25|  India|
|        105| Wei Zhang| wei@example.com| 31|  China|
+-----------+----------+----------------+---+-------+



In [5]:
df_customers.createOrReplaceTempView("customers")
df_orders.createOrReplaceTempView("orders")

In [7]:
spark.sql("""
select order_id, name, amount, 
case when o.amount >= 200 then 'High Value' 
when o.amount >=100 then 'Medium Value'
else 'Low Value' end as order_type
 from customers c inner  join orders o on c.customer_id= o.customer_id
""").show()


+--------+----------+------+------------+
|order_id|      name|amount|  order_type|
+--------+----------+------+------------+
|    5001|  John Doe| 250.5|  High Value|
|    5002|Jane Smith| 145.0|Medium Value|
|    5003|Rita Mehra|389.99|  High Value|
|    5004| Wei Zhang| 89.99|   Low Value|
+--------+----------+------+------------+



In [8]:
df_enriched_op_sql =  spark.sql("""
select order_id, name, amount, 
case when o.amount >= 200 then 'High Value' 
when o.amount >=100 then 'Medium Value'
else 'Low Value' end as order_type
 from customers c inner  join orders o on c.customer_id= o.customer_id
""")


In [9]:
df_enriched_op_sql.show()

+--------+----------+------+------------+
|order_id|      name|amount|  order_type|
+--------+----------+------+------------+
|    5001|  John Doe| 250.5|  High Value|
|    5002|Jane Smith| 145.0|Medium Value|
|    5003|Rita Mehra|389.99|  High Value|
|    5004| Wei Zhang| 89.99|   Low Value|
+--------+----------+------+------------+



In [13]:
df_enriched_op_sql.createOrReplaceTempView("df_enriched_op_sql")
df_enriched_sql_output = spark.sql("""select * from df_enriched_op_sql where
order_type in ('High Value','Medium Value') """)


In [15]:
df_enriched_sql_output.write.mode("overwrite").option("header", True).csv(output_path_csv)